<div style="border: 2px solid #8A9AD0; margin: 1em 0.2em; padding: 0.5em;">

# Single Cell Publication - Data Plotting

by [Helena Rasche](https://training.galaxyproject.org/hall-of-fame/hexylena/)

CC-BY licensed content from the [Galaxy Training Network](https://training.galaxyproject.org/)

**Objectives**


**Objectives**


**Time Estimation: 1H**
</div>


<p>We’ll use some ggplot and tidyverse code to plot the data we collected in part 1</p>
<blockquote class="agenda" style="border: 2px solid #86D486;display: none; margin: 1em 0.2em">
<div class="box-title agenda-title" id="agenda">Agenda</div>
<p>In this tutorial, we will cover:</p>
<ol id="markdown-toc">
<li><a href="#data-cleaning" id="markdown-toc-data-cleaning">Data Cleaning</a></li>
<li><a href="#plot-lines-addedremoved-by-datetime-period" id="markdown-toc-plot-lines-addedremoved-by-datetime-period">Plot: Lines added/removed by date/time period</a></li>
<li><a href="#contributions-over-time" id="markdown-toc-contributions-over-time">Contributions over time</a></li>
<li><a href="#new-x-over-time" id="markdown-toc-new-x-over-time">New X over Time</a></li>
<li><a href="#pageviews" id="markdown-toc-pageviews">Pageviews</a></li>
</ol>
</blockquote>
<p>We’ll use tidyverse (which includes things like magrittr (<code class="language-plaintext highlighter-rouge">%&gt;%</code>) and ggplot2) to load our data. Reshape2 provides the cast/melt functions which can be used to reshape specific datasets into formats that are easier to plot.</p>
<div class="language-R highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit"><span class="n">library</span><span class="p">(</span><span class="n">tidyverse</span><span class="p">)</span><span class="w">
</span><span class="n">library</span><span class="p">(</span><span class="n">reshape2</span><span class="p">)</span><span class="w">
</span><span class="c1"># This path will probably need to be changed depending on where you downloaded that dataset.</span><span class="w">
</span><span class="n">data</span><span class="w"> </span><span class="o">=</span><span class="w"> </span><span class="n">read_tsv</span><span class="p">(</span><span class="s2">"sc.tsv"</span><span class="p">)</span><span class="w">
</span></code></pre></div></div>
<h2 id="data-cleaning">Data Cleaning</h2>
<p>Let’s start by cleaning our data a bit. We’ve got a couple problems with it:</p>
<ul>
<li>there might be some null mergedAt dates, we’ll want to remove those</li>
<li>we probably want to ignore a couple of classes of datasets that were added/removed (images, and the explicit ‘ignore’ type.)</li>
</ul>
<div class="language-R highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit"><span class="n">clean</span><span class="w"> </span><span class="o">=</span><span class="w"> </span><span class="n">data</span><span class="w"> </span><span class="o">%&gt;%</span><span class="w">
</span><span class="n">select</span><span class="p">(</span><span class="n">num</span><span class="p">,</span><span class="w"> </span><span class="n">class</span><span class="p">,</span><span class="w"> </span><span class="n">additions</span><span class="p">,</span><span class="w"> </span><span class="n">deletions</span><span class="p">,</span><span class="w"> </span><span class="n">mergedAt</span><span class="p">)</span><span class="w"> </span><span class="o">%&gt;%</span><span class="w">
</span><span class="n">filter</span><span class="p">(</span><span class="o">!</span><span class="nf">is.na</span><span class="p">(</span><span class="n">mergedAt</span><span class="p">))</span><span class="w"> </span><span class="o">%&gt;%</span><span class="w">
</span><span class="n">group_by</span><span class="p">(</span><span class="n">num</span><span class="p">,</span><span class="w"> </span><span class="n">class</span><span class="p">,</span><span class="w"> </span><span class="n">mergedAt</span><span class="p">,</span><span class="w"> </span><span class="n">month</span><span class="o">=</span><span class="n">floor_date</span><span class="p">(</span><span class="n">mergedAt</span><span class="p">,</span><span class="w"> </span><span class="s1">'month'</span><span class="p">),</span><span class="w"> </span><span class="n">quarter</span><span class="o">=</span><span class="n">floor_date</span><span class="p">(</span><span class="n">mergedAt</span><span class="p">,</span><span class="w"> </span><span class="s1">'quarter'</span><span class="p">))</span><span class="w"> </span><span class="o">%&gt;%</span><span class="w">
</span><span class="n">summarise</span><span class="p">(</span><span class="n">additions</span><span class="o">=</span><span class="nf">sum</span><span class="p">(</span><span class="n">additions</span><span class="p">),</span><span class="w"> </span><span class="n">deletions</span><span class="o">=-</span><span class="nf">sum</span><span class="p">(</span><span class="n">deletions</span><span class="p">))</span><span class="w"> </span><span class="o">%&gt;%</span><span class="w">
</span><span class="n">filter</span><span class="p">(</span><span class="n">class</span><span class="w"> </span><span class="o">!=</span><span class="w"> </span><span class="s2">"ignore"</span><span class="p">)</span><span class="w"> </span><span class="o">%&gt;%</span><span class="w">
</span><span class="n">filter</span><span class="p">(</span><span class="n">class</span><span class="w"> </span><span class="o">!=</span><span class="w"> </span><span class="s2">"image"</span><span class="p">)</span><span class="w"> </span><span class="o">%&gt;%</span><span class="w">
</span><span class="n">arrange</span><span class="p">(</span><span class="n">mergedAt</span><span class="p">)</span><span class="w"> </span><span class="o">%&gt;%</span><span class="w">
</span><span class="n">as_tibble</span><span class="p">()</span><span class="w">
</span></code></pre></div></div>
<p>We’ll setup a ‘theme’ for our plot that mainly consists of using the black and white theme which is quite elegant and readable, and then making some font sizes a wee bit larger:</p>
<div class="language-R highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit"><span class="n">theme</span><span class="w"> </span><span class="o">=</span><span class="w"> </span><span class="n">theme_bw</span><span class="p">()</span><span class="w"> </span><span class="o">+</span><span class="w"> </span><span class="n">theme</span><span class="p">(</span><span class="w">
</span><span class="n">axis.text</span><span class="o">=</span><span class="n">element_text</span><span class="p">(</span><span class="n">size</span><span class="o">=</span><span class="m">14</span><span class="p">),</span><span class="w">
</span><span class="n">plot.title</span><span class="o">=</span><span class="n">element_text</span><span class="p">(</span><span class="n">size</span><span class="o">=</span><span class="m">18</span><span class="p">),</span><span class="w">
</span><span class="n">axis.title</span><span class="o">=</span><span class="n">element_text</span><span class="p">(</span><span class="n">size</span><span class="o">=</span><span class="m">16</span><span class="p">),</span><span class="w">
</span><span class="n">legend.title</span><span class="o">=</span><span class="n">element_text</span><span class="p">(</span><span class="n">size</span><span class="o">=</span><span class="m">14</span><span class="p">),</span><span class="w">
</span><span class="n">legend.text</span><span class="o">=</span><span class="n">element_text</span><span class="p">(</span><span class="n">size</span><span class="o">=</span><span class="m">14</span><span class="p">))</span><span class="w">
</span></code></pre></div></div>
<p>We want all of our plots to look the same which is why we use this trick, it helps us keep a consistent aesthetic without having to re-type the configuration every single time.</p>
<p>Let’s get to plotting!</p>
<h2 id="plot-lines-addedremoved-by-datetime-period">Plot: Lines added/removed by date/time period</h2>
<p>We have a dataset that looks like:</p>
<div class="language-R highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit"><span class="n">clean</span><span class="w"> </span><span class="o">%&gt;%</span><span class="w"> </span><span class="n">head</span><span class="p">(</span><span class="m">10</span><span class="p">)</span><span class="w">
</span></code></pre></div></div>
<p>We’ll want to plot the changes versus the time (either quarter or month), and
then maybe plot them differently based on the class of the addition/removal. So that translates into an aesthetics statement like <code style="color: inherit">aes(x=time, y=additions, fill=class)</code></p>
<p>In ggplot2 you can plot data some different ways, if you provide the datasets upfront, e.g. <code style="color: inherit">data %&gt;% ggplot()</code> you’ll generally do something like <code style="color: inherit">data %&gt;% ggplot(aes(x=a, y=b)) + geom_something()</code> and <code style="color: inherit">geom_something</code> will take the data from the ggplot call. However if you want to plot multiple series, you can also provide the data directly to the <code style="color: inherit">geom_*</code> functions like so:</p>
<div class="language-R highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit"><span class="n">ggplot</span><span class="p">()</span><span class="w"> </span><span class="o">+</span><span class="w">
</span><span class="n">geom_col</span><span class="p">(</span><span class="n">data</span><span class="o">=</span><span class="n">clean</span><span class="p">,</span><span class="w"> </span><span class="n">aes</span><span class="p">(</span><span class="n">x</span><span class="o">=</span><span class="n">quarter</span><span class="p">,</span><span class="w"> </span><span class="n">y</span><span class="o">=</span><span class="n">additions</span><span class="p">,</span><span class="w"> </span><span class="n">fill</span><span class="o">=</span><span class="n">class</span><span class="p">))</span><span class="w"> </span><span class="o">+</span><span class="w">
</span><span class="n">geom_col</span><span class="p">(</span><span class="n">data</span><span class="o">=</span><span class="n">clean</span><span class="p">,</span><span class="w"> </span><span class="n">aes</span><span class="p">(</span><span class="n">x</span><span class="o">=</span><span class="n">quarter</span><span class="p">,</span><span class="w"> </span><span class="n">y</span><span class="o">=</span><span class="n">deletions</span><span class="p">,</span><span class="w"> </span><span class="n">fill</span><span class="o">=</span><span class="n">class</span><span class="p">))</span><span class="w"> </span><span class="o">+</span><span class="w"> </span><span class="n">scale_fill_brewer</span><span class="p">(</span><span class="n">palette</span><span class="w"> </span><span class="o">=</span><span class="w"> </span><span class="s2">"Paired"</span><span class="p">)</span><span class="w"> </span><span class="o">+</span><span class="w">
</span><span class="n">geom_point</span><span class="p">(</span><span class="n">data</span><span class="o">=</span><span class="n">clean</span><span class="p">,</span><span class="w"> </span><span class="n">aes</span><span class="p">(</span><span class="n">x</span><span class="o">=</span><span class="n">mergedAt</span><span class="p">,</span><span class="w"> </span><span class="n">y</span><span class="o">=</span><span class="m">0</span><span class="p">),</span><span class="w"> </span><span class="n">shape</span><span class="o">=</span><span class="m">3</span><span class="p">,</span><span class="w"> </span><span class="n">alpha</span><span class="o">=</span><span class="m">0.3</span><span class="p">,</span><span class="w"> </span><span class="n">color</span><span class="o">=</span><span class="s2">"black"</span><span class="p">)</span><span class="w"> </span><span class="o">+</span><span class="w">
</span><span class="n">theme</span><span class="w"> </span><span class="o">+</span><span class="w">
</span><span class="n">xlab</span><span class="p">(</span><span class="s2">"Quarter"</span><span class="p">)</span><span class="w"> </span><span class="o">+</span><span class="w"> </span><span class="n">ylab</span><span class="p">(</span><span class="s2">"Lines added or removed"</span><span class="p">)</span><span class="w"> </span><span class="o">+</span><span class="w"> </span><span class="n">guides</span><span class="p">(</span><span class="n">fill</span><span class="o">=</span><span class="n">guide_legend</span><span class="p">(</span><span class="n">title</span><span class="o">=</span><span class="s2">"Category"</span><span class="p">))</span><span class="w"> </span><span class="o">+</span><span class="w">
</span><span class="n">ggtitle</span><span class="p">(</span><span class="s2">"Lines added or removed by file type across GTN Single Cell Learning Materials"</span><span class="p">)</span><span class="w">
</span><span class="n">ggsave</span><span class="p">(</span><span class="s2">"sc-lines-by-quarter.png"</span><span class="p">,</span><span class="w"> </span><span class="n">width</span><span class="o">=</span><span class="m">12</span><span class="p">,</span><span class="w"> </span><span class="n">height</span><span class="o">=</span><span class="m">5</span><span class="p">)</span><span class="w">
</span><span class="n">ggplot</span><span class="p">()</span><span class="w"> </span><span class="o">+</span><span class="w">
</span><span class="n">geom_col</span><span class="p">(</span><span class="n">data</span><span class="o">=</span><span class="n">clean</span><span class="p">,</span><span class="w"> </span><span class="n">aes</span><span class="p">(</span><span class="n">x</span><span class="o">=</span><span class="n">month</span><span class="p">,</span><span class="w"> </span><span class="n">y</span><span class="o">=</span><span class="n">additions</span><span class="p">,</span><span class="w"> </span><span class="n">fill</span><span class="o">=</span><span class="n">class</span><span class="p">))</span><span class="w"> </span><span class="o">+</span><span class="w">
</span><span class="n">geom_col</span><span class="p">(</span><span class="n">data</span><span class="o">=</span><span class="n">clean</span><span class="p">,</span><span class="w"> </span><span class="n">aes</span><span class="p">(</span><span class="n">x</span><span class="o">=</span><span class="n">month</span><span class="p">,</span><span class="w"> </span><span class="n">y</span><span class="o">=</span><span class="n">deletions</span><span class="p">,</span><span class="w"> </span><span class="n">fill</span><span class="o">=</span><span class="n">class</span><span class="p">))</span><span class="w"> </span><span class="o">+</span><span class="w"> </span><span class="n">scale_fill_brewer</span><span class="p">(</span><span class="n">palette</span><span class="w"> </span><span class="o">=</span><span class="w"> </span><span class="s2">"Paired"</span><span class="p">)</span><span class="w"> </span><span class="o">+</span><span class="w">
</span><span class="n">theme</span><span class="w"> </span><span class="o">+</span><span class="w">
</span><span class="n">geom_point</span><span class="p">(</span><span class="n">data</span><span class="o">=</span><span class="n">clean</span><span class="p">,</span><span class="w"> </span><span class="n">aes</span><span class="p">(</span><span class="n">x</span><span class="o">=</span><span class="n">mergedAt</span><span class="p">,</span><span class="w"> </span><span class="n">y</span><span class="o">=</span><span class="m">0</span><span class="p">),</span><span class="w"> </span><span class="n">shape</span><span class="o">=</span><span class="m">3</span><span class="p">,</span><span class="w"> </span><span class="n">alpha</span><span class="o">=</span><span class="m">0.3</span><span class="p">,</span><span class="w"> </span><span class="n">color</span><span class="o">=</span><span class="s2">"black"</span><span class="p">)</span><span class="w"> </span><span class="o">+</span><span class="w">
</span><span class="n">xlab</span><span class="p">(</span><span class="s2">"Month"</span><span class="p">)</span><span class="w"> </span><span class="o">+</span><span class="w"> </span><span class="n">ylab</span><span class="p">(</span><span class="s2">"Lines added or removed"</span><span class="p">)</span><span class="w"> </span><span class="o">+</span><span class="w"> </span><span class="n">guides</span><span class="p">(</span><span class="n">fill</span><span class="o">=</span><span class="n">guide_legend</span><span class="p">(</span><span class="n">title</span><span class="o">=</span><span class="s2">"Category"</span><span class="p">))</span><span class="w"> </span><span class="o">+</span><span class="w">
</span><span class="n">ggtitle</span><span class="p">(</span><span class="s2">"Lines added or removed by file type across GTN Single Cell Learning Materials"</span><span class="p">)</span><span class="w">
</span><span class="n">ggsave</span><span class="p">(</span><span class="s2">"sc-lines-by-month.png"</span><span class="p">,</span><span class="w"> </span><span class="n">width</span><span class="o">=</span><span class="m">12</span><span class="p">,</span><span class="w"> </span><span class="n">height</span><span class="o">=</span><span class="m">5</span><span class="p">)</span><span class="w">
</span></code></pre></div></div>
<p><a href="./images/sc-lines-by-quarter.png" rel="noopener noreferrer"><img src="./images/sc-lines-by-quarter.png" alt="very inconsistent lines added by quarter some quarters have more and some have less but it is mostly workflow and then tutorial lines being added or removed." width="3600" height="1500" loading="lazy" /></a></p>
<p>In this case we plotted the data for additions and deletions separately, and we additionally added points based on the actual date of the PRs to visualise their density.</p>
<p>Let’s produce ‘s favourite running sum plots in addition. We’ll start by reshaping our data. Currently we have data that looks like:</p>
<table>
<thead>
<tr>
<th>time</th>
<th>variable</th>
<th>value</th>
</tr>
</thead>
<tbody>
<tr>
<td>today</td>
<td>measure1</td>
<td>1</td>
</tr>
<tr>
<td>today</td>
<td>measure2</td>
<td>10</td>
</tr>
<tr>
<td>yesterday</td>
<td>measure1</td>
<td>30</td>
</tr>
<tr>
<td>yesterday</td>
<td>measure2</td>
<td>5</td>
</tr>
</tbody>
</table>
<p>And we’ll re-shape this to look like this, which will make it easier to calculate changes over the course of a specific series:</p>
<table>
<thead>
<tr>
<th>time</th>
<th>measure1</th>
<th>measure2</th>
</tr>
</thead>
<tbody>
<tr>
<td>today</td>
<td>1</td>
<td>10</td>
</tr>
<tr>
<td>yesterday</td>
<td>30</td>
<td>5</td>
</tr>
</tbody>
</table>
<p>We’ll use the <code style="color: inherit">dcast</code> function to do that:</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">clean %&gt;% select(month, class, additions)  %&gt;% dcast(month ~ class, value.var="additions") %&gt;% head()
</code></pre></div></div>
<p>That doesn’t look quite right so, let’s change how the data is aggregated:</p>
<div class="language-R highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit"><span class="c1"># cumulative</span><span class="w">
</span><span class="n">clean</span><span class="w"> </span><span class="o">%&gt;%</span><span class="w"> </span><span class="n">select</span><span class="p">(</span><span class="n">month</span><span class="p">,</span><span class="w"> </span><span class="n">class</span><span class="p">,</span><span class="w"> </span><span class="n">additions</span><span class="p">)</span><span class="w">  </span><span class="o">%&gt;%</span><span class="w"> </span><span class="n">dcast</span><span class="p">(</span><span class="n">month</span><span class="w"> </span><span class="o">~</span><span class="w"> </span><span class="n">class</span><span class="p">,</span><span class="w"> </span><span class="n">value.var</span><span class="o">=</span><span class="s2">"additions"</span><span class="p">,</span><span class="w"> </span><span class="n">fun.aggregate</span><span class="w"> </span><span class="o">=</span><span class="w"> </span><span class="n">sum</span><span class="p">)</span><span class="w">
</span></code></pre></div></div>
<p>Let’s do it for real now:</p>
<div class="language-R highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit"><span class="n">cumulative</span><span class="w"> </span><span class="o">=</span><span class="w"> </span><span class="n">clean</span><span class="w"> </span><span class="o">%&gt;%</span><span class="w"> </span><span class="n">select</span><span class="p">(</span><span class="n">month</span><span class="p">,</span><span class="w"> </span><span class="n">class</span><span class="p">,</span><span class="w"> </span><span class="n">additions</span><span class="p">)</span><span class="w">  </span><span class="o">%&gt;%</span><span class="w">
</span><span class="n">dcast</span><span class="p">(</span><span class="n">month</span><span class="w"> </span><span class="o">~</span><span class="w"> </span><span class="n">class</span><span class="p">,</span><span class="w"> </span><span class="n">value.var</span><span class="o">=</span><span class="s2">"additions"</span><span class="p">,</span><span class="w"> </span><span class="n">fun.aggregate</span><span class="w"> </span><span class="o">=</span><span class="w"> </span><span class="n">sum</span><span class="p">)</span><span class="w"> </span><span class="o">%&gt;%</span><span class="w">
</span><span class="n">mutate</span><span class="p">(</span><span class="n">across</span><span class="p">(</span><span class="n">bibliography</span><span class="o">:</span><span class="n">workflows</span><span class="p">,</span><span class="w"> </span><span class="n">cumsum</span><span class="p">))</span><span class="w"> </span><span class="o">%&gt;%</span><span class="w">
</span><span class="n">reshape2</span><span class="o">::</span><span class="n">melt</span><span class="p">(</span><span class="n">id.var</span><span class="o">=</span><span class="s2">"month"</span><span class="p">)</span><span class="w">
</span><span class="n">cumulative</span><span class="w"> </span><span class="o">%&gt;%</span><span class="w"> </span><span class="n">ggplot</span><span class="p">(</span><span class="n">aes</span><span class="p">(</span><span class="n">x</span><span class="o">=</span><span class="n">month</span><span class="p">,</span><span class="w"> </span><span class="n">y</span><span class="o">=</span><span class="n">value</span><span class="p">,</span><span class="w"> </span><span class="n">color</span><span class="o">=</span><span class="n">variable</span><span class="p">))</span><span class="w"> </span><span class="o">+</span><span class="w"> </span><span class="n">geom_line</span><span class="p">()</span><span class="w"> </span><span class="o">+</span><span class="w">
</span><span class="n">theme_bw</span><span class="p">()</span><span class="w"> </span><span class="o">+</span><span class="w"> </span><span class="n">theme</span><span class="w"> </span><span class="o">+</span><span class="w">
</span><span class="n">xlab</span><span class="p">(</span><span class="s2">"Month"</span><span class="p">)</span><span class="w"> </span><span class="o">+</span><span class="w"> </span><span class="n">ylab</span><span class="p">(</span><span class="s2">"Lines added"</span><span class="p">)</span><span class="w"> </span><span class="o">+</span><span class="w"> </span><span class="n">guides</span><span class="p">(</span><span class="n">color</span><span class="o">=</span><span class="n">guide_legend</span><span class="p">(</span><span class="n">title</span><span class="o">=</span><span class="s2">"Category"</span><span class="p">))</span><span class="w"> </span><span class="o">+</span><span class="w">
</span><span class="n">ggtitle</span><span class="p">(</span><span class="s2">"Cumulative lines added by category, across GTN Single Cell materials"</span><span class="p">)</span><span class="w">
</span><span class="n">ggsave</span><span class="p">(</span><span class="s2">"sc-lines-cumulative.png"</span><span class="p">,</span><span class="w"> </span><span class="n">width</span><span class="o">=</span><span class="m">12</span><span class="p">,</span><span class="w"> </span><span class="n">height</span><span class="o">=</span><span class="m">8</span><span class="p">)</span><span class="w">
</span></code></pre></div></div>
<p><a href="./images/sc-lines-cumulative.png" rel="noopener noreferrer"><img src="./images/sc-lines-cumulative.png" alt="workflows and tutorials are the highest with on the order of 25k lines added, it used to be quite jagged with code being thrown over the wall but now it's rather smooth with consistent constant updates." width="3600" height="2400" loading="lazy" /></a></p>
<h2 id="contributions-over-time">Contributions over time</h2>
<p>Let’s again start with some cleaning, namely removing all rows with 0 records, and removing future records (at the time of writing.)</p>
<div class="language-R highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit"><span class="n">roles</span><span class="w"> </span><span class="o">=</span><span class="w"> </span><span class="n">read_tsv</span><span class="p">(</span><span class="s2">"sc-roles.tsv"</span><span class="p">)</span><span class="w">
</span><span class="n">w</span><span class="w"> </span><span class="o">=</span><span class="w"> </span><span class="n">roles</span><span class="w"> </span><span class="o">%&gt;%</span><span class="w">
</span><span class="n">filter</span><span class="p">(</span><span class="n">count</span><span class="w"> </span><span class="o">!=</span><span class="w"> </span><span class="m">0</span><span class="p">)</span><span class="w"> </span><span class="o">%&gt;%</span><span class="w">
</span><span class="n">filter</span><span class="p">(</span><span class="o">!</span><span class="n">grepl</span><span class="p">(</span><span class="s2">"2025"</span><span class="p">,</span><span class="w"> </span><span class="n">date</span><span class="p">))</span><span class="w"> </span><span class="o">%&gt;%</span><span class="w">
</span><span class="n">filter</span><span class="p">(</span><span class="o">!</span><span class="n">grepl</span><span class="p">(</span><span class="s2">"2024-12-01"</span><span class="p">,</span><span class="w"> </span><span class="n">date</span><span class="p">))</span><span class="w">
</span><span class="n">w</span><span class="w"> </span><span class="o">%&gt;%</span><span class="w">
</span><span class="n">ggplot</span><span class="p">(</span><span class="n">aes</span><span class="p">(</span><span class="n">x</span><span class="o">=</span><span class="n">date</span><span class="p">,</span><span class="w"> </span><span class="n">y</span><span class="o">=</span><span class="n">count</span><span class="p">,</span><span class="w"> </span><span class="n">color</span><span class="o">=</span><span class="n">area</span><span class="p">))</span><span class="w"> </span><span class="o">+</span><span class="w">
</span><span class="n">theme</span><span class="w"> </span><span class="o">+</span><span class="w">
</span><span class="n">xlab</span><span class="p">(</span><span class="s2">"Date"</span><span class="p">)</span><span class="w"> </span><span class="o">+</span><span class="w"> </span><span class="n">ylab</span><span class="p">(</span><span class="s2">"Unique Contributors"</span><span class="p">)</span><span class="w"> </span><span class="o">+</span><span class="w"> </span><span class="n">guides</span><span class="p">(</span><span class="n">color</span><span class="o">=</span><span class="n">guide_legend</span><span class="p">(</span><span class="n">title</span><span class="o">=</span><span class="s2">"Contribution Area"</span><span class="p">))</span><span class="w"> </span><span class="o">+</span><span class="w">
</span><span class="n">ggtitle</span><span class="p">(</span><span class="s2">"Contributors over time to GTN Single Cell Materials"</span><span class="p">)</span><span class="w"> </span><span class="o">+</span><span class="w">
</span><span class="n">geom_line</span><span class="p">()</span><span class="w">
</span><span class="n">ggsave</span><span class="p">(</span><span class="s2">"sc-contribs.png"</span><span class="p">,</span><span class="w"> </span><span class="n">width</span><span class="o">=</span><span class="m">12</span><span class="p">,</span><span class="w"> </span><span class="n">height</span><span class="o">=</span><span class="m">6</span><span class="p">)</span><span class="w">
</span></code></pre></div></div>
<p><a href="./images/sc-contribs.png" rel="noopener noreferrer"><img src="./images/sc-contribs.png" alt="plot of several lines increasing over time in a very discrete way, authorship remains highest. " width="3600" height="1800" loading="lazy" /></a></p>
<h2 id="new-x-over-time">New X over Time</h2>
<p>Let’s plot all of the new single cell things added over time, all the new FAQs, Tutorials, Slides, Etc:</p>
<div class="language-R highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit"><span class="n">added_by_time</span><span class="w"> </span><span class="o">=</span><span class="w"> </span><span class="n">read_tsv</span><span class="p">(</span><span class="s2">"single-cell-over-time.tsv"</span><span class="p">)</span><span class="w">
</span><span class="n">added_by_time</span><span class="w"> </span><span class="o">%&gt;%</span><span class="w"> </span><span class="n">dcast</span><span class="p">(</span><span class="n">date</span><span class="w"> </span><span class="o">~</span><span class="w"> </span><span class="o">&lt;</span><span class="n">code</span><span class="o">&gt;</span><span class="n">type</span><span class="o">&lt;/</span><span class="n">code</span><span class="o">&gt;</span><span class="p">)</span><span class="w"> </span><span class="o">%&gt;%</span><span class="w">
</span><span class="n">arrange</span><span class="p">(</span><span class="n">date</span><span class="p">)</span><span class="w"> </span><span class="o">%&gt;%</span><span class="w">
</span><span class="n">mutate</span><span class="p">(</span><span class="n">across</span><span class="p">(</span><span class="n">event</span><span class="o">:</span><span class="n">workflow</span><span class="p">,</span><span class="w"> </span><span class="n">cumsum</span><span class="p">))</span><span class="w"> </span><span class="o">%&gt;%</span><span class="w">
</span><span class="n">melt</span><span class="p">(</span><span class="n">id.var</span><span class="o">=</span><span class="s2">"date"</span><span class="p">)</span><span class="w"> </span><span class="o">%&gt;%</span><span class="w">
</span><span class="n">as_tibble</span><span class="p">()</span><span class="w"> </span><span class="o">%&gt;%</span><span class="w"> </span><span class="n">arrange</span><span class="p">(</span><span class="n">date</span><span class="p">)</span><span class="w"> </span><span class="o">%&gt;%</span><span class="w">
</span><span class="n">ggplot</span><span class="p">(</span><span class="n">aes</span><span class="p">(</span><span class="n">x</span><span class="o">=</span><span class="n">date</span><span class="p">,</span><span class="w"> </span><span class="n">y</span><span class="o">=</span><span class="n">value</span><span class="p">,</span><span class="w"> </span><span class="n">color</span><span class="o">=</span><span class="n">variable</span><span class="p">))</span><span class="w"> </span><span class="o">+</span><span class="w"> </span><span class="n">geom_line</span><span class="p">()</span><span class="w"> </span><span class="o">+</span><span class="w">
</span><span class="n">theme_bw</span><span class="p">()</span><span class="w"> </span><span class="o">+</span><span class="w"> </span><span class="n">theme</span><span class="w"> </span><span class="o">+</span><span class="w">
</span><span class="n">xlab</span><span class="p">(</span><span class="s2">"Date"</span><span class="p">)</span><span class="w"> </span><span class="o">+</span><span class="w"> </span><span class="n">ylab</span><span class="p">(</span><span class="s2">"New Single Cell Items"</span><span class="p">)</span><span class="w"> </span><span class="o">+</span><span class="w"> </span><span class="n">guides</span><span class="p">(</span><span class="n">color</span><span class="o">=</span><span class="n">guide_legend</span><span class="p">(</span><span class="n">title</span><span class="o">=</span><span class="s2">"Contribution Type"</span><span class="p">))</span><span class="w"> </span><span class="o">+</span><span class="w">
</span><span class="n">ggtitle</span><span class="p">(</span><span class="s2">"New Single Cell events, FAQs, news, slides, tutorials, videos and workflows in the GTN"</span><span class="p">)</span><span class="w">
</span><span class="n">ggsave</span><span class="p">(</span><span class="s2">"sc-files-cumulative.png"</span><span class="p">,</span><span class="w"> </span><span class="n">width</span><span class="o">=</span><span class="m">6</span><span class="p">,</span><span class="w"> </span><span class="n">height</span><span class="o">=</span><span class="m">6</span><span class="p">)</span><span class="w">
</span></code></pre></div></div>
<p>We may also want to know how many changes there have been since the start date of this study, e.g. October 1st, 2020:</p>
<div class="language-R highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit"><span class="n">since_oct</span><span class="w"> </span><span class="o">=</span><span class="w"> </span><span class="n">added_by_time</span><span class="w"> </span><span class="o">%&gt;%</span><span class="w"> </span><span class="n">dcast</span><span class="p">(</span><span class="n">date</span><span class="w"> </span><span class="o">~</span><span class="w"> </span><span class="o">&lt;</span><span class="n">code</span><span class="o">&gt;</span><span class="n">type</span><span class="o">&lt;/</span><span class="n">code</span><span class="o">&gt;</span><span class="p">)</span><span class="w"> </span><span class="o">%&gt;%</span><span class="w">
</span><span class="n">arrange</span><span class="p">(</span><span class="n">date</span><span class="p">)</span><span class="w"> </span><span class="o">%&gt;%</span><span class="w">
</span><span class="n">mutate</span><span class="p">(</span><span class="n">across</span><span class="p">(</span><span class="n">event</span><span class="o">:</span><span class="n">workflow</span><span class="p">,</span><span class="w"> </span><span class="n">cumsum</span><span class="p">))</span><span class="w"> </span><span class="o">%&gt;%</span><span class="w">
</span><span class="n">filter</span><span class="p">(</span><span class="n">date</span><span class="w"> </span><span class="o">&gt;</span><span class="w"> </span><span class="n">as.Date</span><span class="p">(</span><span class="s2">"2020-10-01"</span><span class="p">))</span><span class="w">
</span><span class="c1"># Pull out the first/last date as our start ane dnwend</span><span class="w">
</span><span class="n">start_date</span><span class="w"> </span><span class="o">=</span><span class="w"> </span><span class="p">(</span><span class="n">since_oct</span><span class="w"> </span><span class="o">%&gt;%</span><span class="w"> </span><span class="n">head</span><span class="p">(</span><span class="n">n</span><span class="o">=</span><span class="m">1</span><span class="p">))</span><span class="o">&#36;</span><span class="n">date</span><span class="w">
</span><span class="n">end_date</span><span class="w"> </span><span class="o">=</span><span class="w"> </span><span class="p">(</span><span class="n">since_oct</span><span class="w"> </span><span class="o">%&gt;%</span><span class="w"> </span><span class="n">tail</span><span class="p">(</span><span class="n">n</span><span class="o">=</span><span class="m">1</span><span class="p">))</span><span class="o">&#36;</span><span class="n">date</span><span class="w">
</span><span class="c1"># Table of our changes.</span><span class="w">
</span><span class="n">since_oct</span><span class="w"> </span><span class="o">%&gt;%</span><span class="w">
</span><span class="n">filter</span><span class="p">(</span><span class="n">date</span><span class="w"> </span><span class="o">==</span><span class="w"> </span><span class="n">start_date</span><span class="w"> </span><span class="o">|</span><span class="w"> </span><span class="n">date</span><span class="w"> </span><span class="o">==</span><span class="w"> </span><span class="n">end_date</span><span class="p">)</span><span class="w"> </span><span class="o">%&gt;%</span><span class="w">  </span><span class="c1"># Just those rows</span><span class="w">
</span><span class="n">mutate</span><span class="p">(</span><span class="n">date</span><span class="w"> </span><span class="o">=</span><span class="w"> </span><span class="n">case_when</span><span class="p">(</span><span class="n">date</span><span class="w"> </span><span class="o">==</span><span class="w"> </span><span class="n">start_date</span><span class="w"> </span><span class="o">~</span><span class="w"> </span><span class="s1">'start'</span><span class="p">,</span><span class="w"> </span><span class="n">date</span><span class="w"> </span><span class="o">==</span><span class="w"> </span><span class="n">end_date</span><span class="w"> </span><span class="o">~</span><span class="w"> </span><span class="s1">'end'</span><span class="p">))</span><span class="w"> </span><span class="o">%&gt;%</span><span class="w"> </span><span class="c1"># Relabel the start/end as literal string start and end</span><span class="w">
</span><span class="n">pivot_longer</span><span class="p">(</span><span class="o">-</span><span class="n">date</span><span class="p">)</span><span class="w"> </span><span class="o">%&gt;%</span><span class="w"> </span><span class="n">pivot_wider</span><span class="p">(</span><span class="n">names_from</span><span class="o">=</span><span class="n">date</span><span class="p">,</span><span class="w"> </span><span class="n">values_from</span><span class="o">=</span><span class="n">value</span><span class="p">)</span><span class="w"> </span><span class="o">%&gt;%</span><span class="w"> </span><span class="c1"># Transpose the data</span><span class="w">
</span><span class="n">mutate</span><span class="p">(</span><span class="n">increase</span><span class="o">=</span><span class="n">end</span><span class="w"> </span><span class="o">-</span><span class="w"> </span><span class="n">start</span><span class="p">)</span><span class="w"> </span><span class="o">%&gt;%</span><span class="w"> </span><span class="c1"># And calculate our increase</span><span class="w">
</span><span class="n">select</span><span class="p">(</span><span class="n">name</span><span class="p">,</span><span class="w"> </span><span class="n">increase</span><span class="p">)</span><span class="w"> </span><span class="o">%&gt;%</span><span class="w"> </span><span class="n">arrange</span><span class="p">(</span><span class="o">-</span><span class="n">increase</span><span class="p">)</span><span class="w">
</span></code></pre></div></div>
<h2 id="pageviews">Pageviews</h2>
<p>GTN uses the Galaxy Europe Plausible server for collecting metrics (you can change your preferences in <a href="{% link user/privacy.md %}">your GTN privacy preferences</a>).
We can download the data from the server and plot it, filtering by our preferred start/end dates and filters (namely that page includes <code style="color: inherit">/topics/single-cell/</code>).
Unfortunately the data is downloaded as a zip file which we’ll then need to extract data from:</p>
<div class="language-R highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit"><span class="n">system</span><span class="p">(</span><span class="s2">"wget 'https://plausible.galaxyproject.eu/training.galaxyproject.org/export?period=custom&amp;date=2024-11-07&amp;from=2022-08-01&amp;to=2024-11-07&amp;filters=%7B%22page%22%3A%22~%2Ftopics%2Fsingle-cell%2F%22%7D&amp;with_imported=true&amp;interval=date' -O sc-stats.zip"</span><span class="p">)</span><span class="w">
</span></code></pre></div></div>
<p>We can use the unzip function to read a single file directly from the zip:</p>
<div class="language-R highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit"><span class="n">views</span><span class="w"> </span><span class="o">=</span><span class="w"> </span><span class="n">read_csv</span><span class="p">(</span><span class="n">unzip</span><span class="p">(</span><span class="s2">"sc-stats.zip"</span><span class="p">,</span><span class="w"> </span><span class="s2">"visitors.csv"</span><span class="p">))</span><span class="w">
</span></code></pre></div></div>
<p>With that we’re ready to plot. We’re going to use a new feature for our plot,
<code class="language-plaintext highlighter-rouge">annotate</code>. Annotation allows you to draw arbitrary features atop your plot, in
this case we’re going to draw rectangles to indicate outages and events that
might have affected our data.</p>
<div class="language-R highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit"><span class="n">y</span><span class="w"> </span><span class="o">=</span><span class="w"> </span><span class="m">3300</span><span class="w">
</span><span class="n">xoff</span><span class="w"> </span><span class="o">=</span><span class="w"> </span><span class="m">3</span><span class="w">
</span><span class="n">views</span><span class="w"> </span><span class="o">%&gt;%</span><span class="w"> </span><span class="n">group_by</span><span class="p">(</span><span class="n">date</span><span class="o">=</span><span class="n">floor_date</span><span class="p">(</span><span class="n">date</span><span class="p">,</span><span class="w"> </span><span class="s1">'week'</span><span class="p">))</span><span class="w"> </span><span class="o">%&gt;%</span><span class="w">
</span><span class="n">summarise</span><span class="p">(</span><span class="n">date</span><span class="p">,</span><span class="w"> </span><span class="n">visitors</span><span class="o">=</span><span class="nf">sum</span><span class="p">(</span><span class="n">visitors</span><span class="p">),</span><span class="w"> </span><span class="n">pageviews</span><span class="o">=</span><span class="nf">sum</span><span class="p">(</span><span class="n">pageviews</span><span class="p">))</span><span class="w"> </span><span class="o">%&gt;%</span><span class="w">
</span><span class="n">filter</span><span class="p">(</span><span class="n">visitors</span><span class="w"> </span><span class="o">!=</span><span class="w"> </span><span class="m">0</span><span class="w"> </span><span class="o">|</span><span class="w"> </span><span class="n">pageviews</span><span class="w"> </span><span class="o">!=</span><span class="w"> </span><span class="m">0</span><span class="p">)</span><span class="w"> </span><span class="o">%&gt;%</span><span class="w">
</span><span class="n">melt</span><span class="p">(</span><span class="n">id.var</span><span class="o">=</span><span class="s2">"date"</span><span class="p">)</span><span class="w">  </span><span class="o">%&gt;%</span><span class="w">
</span><span class="n">ggplot</span><span class="p">(</span><span class="n">aes</span><span class="p">(</span><span class="n">x</span><span class="o">=</span><span class="n">date</span><span class="p">,</span><span class="w"> </span><span class="n">y</span><span class="o">=</span><span class="n">value</span><span class="p">,</span><span class="w"> </span><span class="n">color</span><span class="o">=</span><span class="n">variable</span><span class="p">))</span><span class="w"> </span><span class="o">+</span><span class="w"> </span><span class="n">geom_line</span><span class="p">()</span><span class="w"> </span><span class="o">+</span><span class="w">
</span><span class="n">annotate</span><span class="p">(</span><span class="s2">"rect"</span><span class="p">,</span><span class="w"> </span><span class="n">xmin</span><span class="w"> </span><span class="o">=</span><span class="w"> </span><span class="n">as.Date</span><span class="p">(</span><span class="s2">"2023-10-01"</span><span class="p">),</span><span class="w"> </span><span class="n">xmax</span><span class="w"> </span><span class="o">=</span><span class="w"> </span><span class="n">as.Date</span><span class="p">(</span><span class="s2">"2023-10-27"</span><span class="p">),</span><span class="w"> </span><span class="n">ymin</span><span class="w"> </span><span class="o">=</span><span class="w"> </span><span class="m">0</span><span class="p">,</span><span class="w"> </span><span class="n">ymax</span><span class="w"> </span><span class="o">=</span><span class="w"> </span><span class="n">y</span><span class="p">,</span><span class="w">  </span><span class="n">alpha</span><span class="w"> </span><span class="o">=</span><span class="w"> </span><span class="m">.2</span><span class="p">)</span><span class="w"> </span><span class="o">+</span><span class="w"> </span><span class="c1"># Outage</span><span class="w">
</span><span class="n">annotate</span><span class="p">(</span><span class="s2">"rect"</span><span class="p">,</span><span class="w"> </span><span class="n">xmin</span><span class="w"> </span><span class="o">=</span><span class="w"> </span><span class="n">as.Date</span><span class="p">(</span><span class="s2">"2023-05-22"</span><span class="p">)</span><span class="w"> </span><span class="o">-</span><span class="w"> </span><span class="n">xoff</span><span class="p">,</span><span class="w"> </span><span class="n">xmax</span><span class="w"> </span><span class="o">=</span><span class="w"> </span><span class="n">as.Date</span><span class="p">(</span><span class="s2">"2023-05-26"</span><span class="p">)</span><span class="w"> </span><span class="o">-</span><span class="w"> </span><span class="n">xoff</span><span class="p">,</span><span class="w"> </span><span class="n">ymin</span><span class="w"> </span><span class="o">=</span><span class="w"> </span><span class="m">0</span><span class="p">,</span><span class="w"> </span><span class="n">ymax</span><span class="w"> </span><span class="o">=</span><span class="w"> </span><span class="n">y</span><span class="p">,</span><span class="w">  </span><span class="n">alpha</span><span class="w"> </span><span class="o">=</span><span class="w"> </span><span class="m">.2</span><span class="p">)</span><span class="w"> </span><span class="o">+</span><span class="w"> </span><span class="c1"># Smorg3</span><span class="w">
</span><span class="n">annotate</span><span class="p">(</span><span class="s2">"rect"</span><span class="p">,</span><span class="w"> </span><span class="n">xmin</span><span class="w"> </span><span class="o">=</span><span class="w"> </span><span class="n">as.Date</span><span class="p">(</span><span class="s2">"2024-10-07"</span><span class="p">)</span><span class="w"> </span><span class="o">-</span><span class="w"> </span><span class="n">xoff</span><span class="p">,</span><span class="w"> </span><span class="n">xmax</span><span class="w"> </span><span class="o">=</span><span class="w"> </span><span class="n">as.Date</span><span class="p">(</span><span class="s2">"2024-10-11"</span><span class="p">)</span><span class="w"> </span><span class="o">-</span><span class="w"> </span><span class="n">xoff</span><span class="p">,</span><span class="w"> </span><span class="n">ymin</span><span class="w"> </span><span class="o">=</span><span class="w"> </span><span class="m">0</span><span class="p">,</span><span class="w"> </span><span class="n">ymax</span><span class="w"> </span><span class="o">=</span><span class="w"> </span><span class="n">y</span><span class="p">,</span><span class="w">  </span><span class="n">alpha</span><span class="w"> </span><span class="o">=</span><span class="w"> </span><span class="m">.2</span><span class="p">)</span><span class="w"> </span><span class="o">+</span><span class="w"> </span><span class="c1"># GTA</span><span class="w">
</span><span class="n">annotate</span><span class="p">(</span><span class="s2">"rect"</span><span class="p">,</span><span class="w"> </span><span class="n">xmin</span><span class="w"> </span><span class="o">=</span><span class="w"> </span><span class="n">as.Date</span><span class="p">(</span><span class="s2">"2024-09-16"</span><span class="p">)</span><span class="w"> </span><span class="o">-</span><span class="w"> </span><span class="n">xoff</span><span class="p">,</span><span class="w"> </span><span class="n">xmax</span><span class="w"> </span><span class="o">=</span><span class="w"> </span><span class="n">as.Date</span><span class="p">(</span><span class="s2">"2024-09-20"</span><span class="p">)</span><span class="w"> </span><span class="o">-</span><span class="w"> </span><span class="n">xoff</span><span class="p">,</span><span class="w"> </span><span class="n">ymin</span><span class="w"> </span><span class="o">=</span><span class="w"> </span><span class="m">0</span><span class="p">,</span><span class="w"> </span><span class="n">ymax</span><span class="w"> </span><span class="o">=</span><span class="w"> </span><span class="n">y</span><span class="p">,</span><span class="w">  </span><span class="n">alpha</span><span class="w"> </span><span class="o">=</span><span class="w"> </span><span class="m">.2</span><span class="p">)</span><span class="w"> </span><span class="o">+</span><span class="w"> </span><span class="c1"># Bootcamp</span><span class="w">
</span><span class="n">theme_bw</span><span class="p">()</span><span class="w"> </span><span class="o">+</span><span class="w"> </span><span class="n">theme</span><span class="w"> </span><span class="o">+</span><span class="w">
</span><span class="n">scale_y_continuous</span><span class="p">(</span><span class="n">expand</span><span class="w"> </span><span class="o">=</span><span class="w"> </span><span class="nf">c</span><span class="p">(</span><span class="m">0</span><span class="p">,</span><span class="w"> </span><span class="m">0</span><span class="p">),</span><span class="w"> </span><span class="n">limits</span><span class="w"> </span><span class="o">=</span><span class="w"> </span><span class="nf">c</span><span class="p">(</span><span class="m">0</span><span class="p">,</span><span class="w"> </span><span class="kc">NA</span><span class="p">))</span><span class="w"> </span><span class="o">+</span><span class="w">
</span><span class="n">xlab</span><span class="p">(</span><span class="s2">"Date"</span><span class="p">)</span><span class="w"> </span><span class="o">+</span><span class="w"> </span><span class="n">ylab</span><span class="p">(</span><span class="s2">"Count"</span><span class="p">)</span><span class="w"> </span><span class="o">+</span><span class="w"> </span><span class="n">guides</span><span class="p">(</span><span class="n">color</span><span class="o">=</span><span class="n">guide_legend</span><span class="p">(</span><span class="n">title</span><span class="o">=</span><span class="s2">"Metric"</span><span class="p">))</span><span class="w"> </span><span class="o">+</span><span class="w">
</span><span class="n">ggtitle</span><span class="p">(</span><span class="s2">"GTN Single Cell Visits"</span><span class="p">)</span><span class="w">
</span><span class="n">ggsave</span><span class="p">(</span><span class="s2">"sc-pageviews.png"</span><span class="p">,</span><span class="w"> </span><span class="n">width</span><span class="o">=</span><span class="m">14</span><span class="p">,</span><span class="w"> </span><span class="n">height</span><span class="o">=</span><span class="m">4</span><span class="p">)</span><span class="w">
</span></code></pre></div></div>
<p>for our lovely pageview plot!</p>
<p><a href="./images/sc-pageviews.png" rel="noopener noreferrer"><img src="./images/sc-pageviews.png" alt="pageviews over time, it's relatively stable except for the large annotated outage, and a large spike around the bootcamp date. " width="4200" height="1200" loading="lazy" /></a></p>
<blockquote class="tip" style="border: 2px solid #FFE19E; margin: 1em 0.2em">
<div class="box-title tip-title" id="tip-do-you-want-to-re-do-this-analysis-for-your-community"><button class="gtn-boxify-button tip" type="button" aria-controls="tip-do-you-want-to-re-do-this-analysis-for-your-community" aria-expanded="true"><i class="far fa-lightbulb" aria-hidden="true" ></i> <span>Tip: Do you want to re-do this analysis? For your community?</span><span class="fold-unfold fa fa-minus-square"></span></button></div>
<p><a href="https://github.com/galaxyproject/training-material/issues/new?title=GTN%20Metadata%20reuse">Discuss this with us</a> and we can perhaps generalise this analysis, to reduce the amount of data processing you need to do, and make it more accessible for everyone!</p>
</blockquote>


# Key Points


# Congratulations on successfully completing this tutorial!

Please [fill out the feedback on the GTN website](https://training.galaxyproject.org/training-material/topics/contributing/tutorials/meta-analysis-plot/tutorial.html#feedback) and check there for further resources!
